# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [93]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [43]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,25.48,74,69,2.58,PN,1739146040
1,1,edinburgh of the seven seas,-37.0676,-12.3116,15.82,75,7,4.13,SH,1739146039
2,2,east ballina,-28.8667,153.5833,26.51,59,27,4.20,AU,1739146104
3,3,margaret river,-33.9500,115.0667,21.18,75,88,4.50,AU,1739146105
4,4,massawa,15.6097,39.4500,24.22,76,83,3.16,ER,1739146106


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [45]:
%%capture --no-display

# Configure the map plot
city_data_df.hvplot.points(
    'Lng',  
    'Lat',  
    geo=True,  
    tiles='OSM',  
    size=10,  
    color='City',  
    hover_cols=['City', 'Country']  
)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [47]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df[
    (city_data_df['Max Temp'] > 21) &
    (city_data_df['Max Temp'] < 27) &
    (city_data_df['Wind Speed'] < 4.5) &
    (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,14,cabo san lucas,22.8909,-109.9124,25.17,25,0,2.57,MX,1739146117
32,32,rio grande,-32.0350,-52.0986,23.91,84,0,4.01,BR,1739146138
40,40,puerto maldonado,-12.6000,-69.1833,24.22,94,0,1.03,PE,1739146149
52,52,wailua homesteads,22.0669,-159.3780,25.63,64,0,3.09,US,1739146164
64,64,lihue,21.9789,-159.3672,25.97,61,0,3.09,US,1739146178
163,163,kapa'a,22.0752,-159.3190,26.38,64,0,3.09,US,1739146295
272,272,petrolina,-9.3986,-40.5008,25.03,78,0,0.00,BR,1739146292
388,388,nombre de dios,23.8500,-104.2333,26.66,14,0,3.14,MX,1739146573
512,512,san rafael del yuma,18.4299,-68.6739,23.11,82,0,4.25,DO,1739146715


### Step 3: Create a new DataFrame called `hotel_df`.

In [49]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df [['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
14,cabo san lucas,MX,22.8909,-109.9124,25,
32,rio grande,BR,-32.0350,-52.0986,84,
40,puerto maldonado,PE,-12.6000,-69.1833,94,
52,wailua homesteads,US,22.0669,-159.3780,64,
64,lihue,US,21.9789,-159.3672,61,
163,kapa'a,US,22.0752,-159.3190,64,
272,petrolina,BR,-9.3986,-40.5008,78,
388,nombre de dios,MX,23.8500,-104.2333,14,
512,san rafael del yuma,DO,18.4299,-68.6739,82,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [65]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories':'accommodation.hotel',
    'limit' : 1,
    'apiKey' : geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row ['Lat']
    lng = row ['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params).json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cabo san lucas - nearest hotel: Comfort Rooms
rio grande - nearest hotel: Hotel Vila Moura Executivo
puerto maldonado - nearest hotel: Puerto Palmeras
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
lihue - nearest hotel: Kauai Palms
kapa'a - nearest hotel: Pono Kai Resort
petrolina - nearest hotel: Neuman
nombre de dios - nearest hotel: No hotel found
san rafael del yuma - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
14,cabo san lucas,MX,22.8909,-109.9124,25,Comfort Rooms
32,rio grande,BR,-32.0350,-52.0986,84,Hotel Vila Moura Executivo
40,puerto maldonado,PE,-12.6000,-69.1833,94,Puerto Palmeras
52,wailua homesteads,US,22.0669,-159.3780,64,Hilton Garden Inn Kauai Wailua Bay
64,lihue,US,21.9789,-159.3672,61,Kauai Palms
163,kapa'a,US,22.0752,-159.3190,64,Pono Kai Resort
272,petrolina,BR,-9.3986,-40.5008,78,Neuman
388,nombre de dios,MX,23.8500,-104.2333,14,No hotel found
512,san rafael del yuma,DO,18.4299,-68.6739,82,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [91]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng',  
    'Lat',  
    geo=True,  
    tiles='OSM',  
    size=100,  
    color='City',  
    hover_cols=['City', 'Country', 'Hotel Name'],  
    title="Cities with Nearby Hotels",
    width = 1200,
    height = 800
)
# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)